# Table of Contents
* [Intro](#Intro)
* [Load Data](#Load-Data)
* [Generate and Save Stats](#Generate-and-Save-Stats)
* [Basic Stats](#Basic-Stats)
	* [Preliminary Stats](#Preliminary-Stats)
	* [Weekday Stats](#Weekday-Stats)
	* [Monthly Stats](#Monthly-Stats)
	* [Year-Month Stats](#Year-Month-Stats)
	* [Weekday Stats By Month](#Weekday-Stats-By-Month)
	* [Daily Stats](#Daily-Stats)
* [Intraday Stats](#Intraday-Stats)
	* [Daily Stats](#Daily-Stats)


# Intro

This notebook explores analysis and visualization of Fitbit sleep data.

In [ ]:
import os
import sys
from os.path import join

import pandas as pd
import numpy as np
import seaborn as sns
import time
import datetime
import matplotlib.pyplot as plt

sys.path.append(os.path.join(os.getcwd(), "src"))
from resources import RESOURCE_PATH
from stats import sleepStats, combinedStats
from util import utils, plotting as mplot

%load_ext autoreload
%autoreload 2

%matplotlib notebook
sns.set_context("paper")

dataFolder = "path_to_your_fitbit_JSON_export"
statsFolder = join(dataFolder, os.path.pardir, 'folder_name_for_generated_stats')

In [ ]:
import plotly.plotly as py
from plotly.offline import init_notebook_mode, enable_mpl_offline, iplot_mpl
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline(connected=True)
enable_mpl_offline()

from ipywidgets import interact, widgets
#from IPython.display import display, clear_output

In [ ]:
# Enable logging from Fitbit Analyer code
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
logger.handlers[0].stream = sys.stdout

# Load Data

In [ ]:
# helper method to load provided test data
def load_test_sleep_data():
    filepath =  RESOURCE_PATH + "\\unittest\\test_sleep_basic01.csv"
    data1 = utils.loadIntradayData(filepath)
    filepath =  RESOURCE_PATH + "\\unittest\\test_sleep_basic02.csv"
    data2 = utils.loadIntradayData(filepath)
    return [data1, data2]

Load sleep data from raw JSON export of Fitbit records.

In [ ]:
start = time.time()
#sleepData = utils.loadSleepData(dataFolder) # use for loading your own data
sleepData = load_test_sleep_data() #use for testing
end = time.time()
print("Data loaded in {:.2f}s".format(end - start))
print("Loaded {} dataframes".format(len(sleepData)))
print("{} total entries".format(np.sum([df.size for df in sleepData])))
print("Sample from first dataframe:")
print(sleepData[0].head())

# Generate and Save Stats

For the loaded sleep data generate all currently avaiable stats:
* **Basic Stats** (sleep values count, sleep efficiency, hours of sleep, total minutes in bed, N max-values for each stat)
* **Timing Stats** (first minute asleep, to bed time, wake up time, sleep interval min/max length)
* **Intervals Stats** for each day all the sleep intervals lengths
* **Intraday Stats** minute to minute report for each day, for the specified sleep values. Total value count, with normalization and centering on specific time.

In [ ]:
start = time.time()
basicAndTimingStats = sleepStats.generateStatsFrom(sleepData, sleepStats.STATS_NAME_BASIC_AND_TIMING)
end = time.time()
print("Computed basicAndTimingStats in {:.2f}s".format(end - start))
start = time.time()
intervalsStats = sleepStats.generateStatsFrom(sleepData, sleepStats.STATS_NAME_INTERVALS)
end = time.time()
print("Computed intervalsStats in {:.2f}s".format(end - start))
start = time.time()
intradayStats = sleepStats.generateStatsFrom(sleepData, sleepStats.STATS_NAME_INTRADAY)
end = time.time()
print("Computed intradayStats in {:.2f}s".format(end - start))

In [ ]:
#print(basicAndTimingStats.head())
#print(intervalsStats.head())
#print(intradayStats.head())

Save generated stats to file

In [ ]:
today = datetime.date.today().strftime("%Y_%m_%d")
basicAndTimingStatsFilepath = os.path.join(statsFolder, "basicAndTimingStats_{}.csv".format(today))
intervalsStatsFilepath = os.path.join(statsFolder, "intervalStats_{}.csv".format(today))
intradayStatsFilepath = os.path.join(statsFolder, "intradayStats_{}.csv".format(today))

basicAndTimingStats.to_csv(basicAndTimingStatsFilepath, index=False)
intervalsStats.to_csv(intervalsStatsFilepath, index=False)
intradayStats.to_csv(intradayStatsFilepath, index=False)

# Basic Stats

In [ ]:
# load basic and timing stats
basicAndTimingStatsFilename = "basicAndTimingStats_{}.csv".format(today)
basicAndTimingStats = pd.read_csv(os.path.join(statsFolder, basicAndTimingStatsFilename), 
                    parse_dates=['date', 'to_bed_time', 'wake_up_time'])
basicAndTimingStats.head()

In [ ]:
basicAndTimingStats.info()

## Preliminary Stats

In [ ]:
# plot preliminary stats (static plot) 
plot = mplot.plotPreliminaryStats(basicAndTimingStats)

## Weekday Stats

In [ ]:
# plot preliminary stats (static plot) 
plot = mplot.plotWeekdayStatsSleep(basicAndTimingStats)
plot

In [ ]:
# transform static plot using Plotly
iplot_mpl(plot.fig)

## Monthly Stats

In [ ]:
plot = mplot.plotMonthlyStatsSleep(basicAndTimingStats)
plot

## Year-Month Stats

In [ ]:
# plot Year-Month stats (static plot)
plot = mplot.plotYearAndMonthStatsSleep(basicAndTimingStats)

In [ ]:
# interactive single year-month stat
@interact(stat_name=mplot.NAMES.keys())
def iplot_yearAndMonthStats(stat_name):
    data = basicAndTimingStats.groupby(basicAndTimingStats['date'].dt.to_period("M"))[[stat_name]].mean()
    data.iplot(title='Year-Month Average - {}'.format(mplot.NAMES[stat_name]))

## Weekday Stats By Month

In [ ]:
# plot weekday stats by month (static plot) 
plot = mplot.plotWeekdayStatsByMonthSleep(basicAndTimingStats)

In [ ]:
weekdayByMonthStats = mplot._prepareWeekdayByMonthStats(basicAndTimingStats)

In [ ]:
@interact(stat_name=mplot.NAMES.keys())
def iplot_weekdayByMonthStats(stat_name):
    weekdayByMonthStats.pivot('day', 'month', values=stat_name).iplot(title='Weekday By Month - {}'.format(mplot.NAMES[stat_name]),
                                                    xTitle='Weekday', yTitle=mplot.NAMES[stat_name])

## Daily Stats

In [ ]:
@interact(stat_name=mplot.NAMES.keys())
def iplot_weekdayByMonthStats(stat_name):
    data = basicAndTimingStats[['date', stat_name]].set_index(['date'])
    data.iplot(title='Daily Stats - {}'.format(mplot.NAMES[stat_name]), yTitle=mplot.NAMES[stat_name])

# Intraday Stats

In [ ]:
intradayStatsFilename = "intradayStats_{}.csv".format(today)
intradayStats = pd.read_csv(os.path.join(statsFolder, intradayStats), 
                    parse_dates=['date', 'to_bed_time', 'wake_up_time'])
intradayStats.drop("date", axis=1, inplace=True)
data = intradayStats.apply(pd.value_counts)
#mplot.plotSleepValueHeatmap(data, sleepValue=1)

In [ ]:
normIntradayCountStats = sleepStats.normalizedIntradayCountStats(intradayStats)
centeredIntradayCountStats = sleepStats.centerIntradayCountStats(normIntradayCountStats)
#mplot.plotSleepValueHeatmap(centeredIntradayCountStats, sleepValue=1)

## Daily Stats


In [ ]:
#stats.set_index('date', inplace=True)
stats['sleep_efficiency_rol_mean'] = stats['sleep_efficiency'].rolling(center=False,window=20).mean()
stats['sleep_efficiency'].plot()
stats['sleep_efficiency_rol_mean'].plot()
sns.plt.show()

In [ ]:
testData = stats['restless']
testData.resample('20D').mean().plot()
testData.plot()
sns.plt.show()